In [40]:
from tensorflow.keras.layers import Embedding,Input,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
from tqdm import tqdm
import pandas as pd
import numpy as np
from nltk import FreqDist
from string import punctuation as pt

In [41]:

# 텍스트 데이터 가져오기.
train_df=pd.read_table('./data/ratings_train.txt')
test_df=pd.read_table('./data/ratings_test.txt')

In [42]:
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [43]:
train_df.document = train_df.document.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",regex=True)  #한글을 제외한 단어는 정규식으로 제거

In [44]:
train_df=train_df['document']   

In [45]:
train_df.isna().sum()# 결측치의 수 확인

5

In [46]:
train_df.dropna(inplace=True)   #결측치 제거

In [47]:
# 데이터셋 사전 관련 상수
NUM_SIZE=10000
OOV='<UNK>'

tokenizer=Tokenizer(num_words=NUM_SIZE,oov_token=OOV)

In [48]:
# sentences 데이터 기반 단어 사전 생성
tokenizer.fit_on_texts(train_df)

In [49]:
tokenizer.word_counts, tokenizer.word_index

(OrderedDict([('아', 1664),
              ('더빙', 126),
              ('진짜', 6213),
              ('짜증나네요', 17),
              ('목소리', 108),
              ('흠포스터보고', 1),
              ('초딩영화줄오버연기조차', 1),
              ('가볍지', 17),
              ('않구나', 2),
              ('너무재밓었다그래서보는것을추천한다', 1),
              ('교도소', 4),
              ('이야기구먼', 1),
              ('솔직히', 943),
              ('재미는', 292),
              ('없다평점', 1),
              ('조정', 11),
              ('사이몬페그의', 1),
              ('익살스런', 2),
              ('연기가', 743),
              ('돋보였던', 23),
              ('영화스파이더맨에서', 1),
              ('늙어보이기만', 1),
              ('했던', 138),
              ('커스틴', 4),
              ('던스트가', 1),
              ('너무나도', 193),
              ('이뻐보였다', 1),
              ('막', 192),
              ('걸음마', 1),
              ('뗀', 2),
              ('세부터', 1),
              ('초등학교', 52),
              ('학년생인', 1),
              ('살용영화ㅋㅋㅋ별반개도', 1),
              ('아까움', 252),
             

In [50]:
stopwords = ['너무','다','적','진짜','에서','점','영화','정말','의','가',
                    '이','은','들','는','좀','잘','걍','과','도','를','으로','자',
                    '에','와','한','하다','것','그','안','인','이런','내','못','게',
                    '이다','고','더','때','거','요','본','지','네','입니다','있는',
                    '걸','이렇게','임','라','인데','듯','기','서','난','면','대','라고',
                    '보면','이나','때문','해서','된','엔','두','이고','랑','전','애',
                    '없이','해','속','한다','라는','뿐','되는','니','줄','냐','하게',
                    '하고','합니다','번','하지','보고','이건','그냥','왜','ㅋㅋ','이거',
                    '이게','그리고','게다가','수','영화를','영화가','영화는','참','아','역시',
                    '다시','나옴','또','꼭','하는','없고','ㅋㅋㅋ','ㅠㅠ','어제','봤는데','없는','없다',
                    '보는','많이','뭐','무슨','ㅋ','암튼','ㅎ','ㅎㅎ','ㅎㅎㅎ','ㅡㅡ','내가','볼','대한','만든','벌써',
                    '전혀','말이','봐도','넘','솔직히','재밌게','가장','같은','하나의']

In [51]:
#토큰화


# KoNLPy의 Okt를 사용하여 토큰화
from tqdm import tqdm

# 현재 텍스트를 분리 => 공백기준
my_words=[]
for st in tqdm(train_df):
    ret=st.split()
    for _ in ret:
        if _ not in stopwords and _ not in pt:
            my_words.append(_)

100%|██████████| 149995/149995 [00:01<00:00, 104299.75it/s]


In [52]:
# 토큰화 데이터를 기반, 단어집합을 생성.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(my_words)

In [53]:
# 전체 단어 갯수
word_index=len(tokenizer.word_index)
word_index

299468

In [54]:
# 생성 단어집합 출력.
tokenizer.word_index

{'최고의': 1,
 '드라마': 2,
 '완전': 3,
 '평점': 4,
 '좋은': 5,
 '평점이': 6,
 '연기': 7,
 '쓰레기': 8,
 '최고': 9,
 '스토리': 10,
 '아깝다': 11,
 '마지막': 12,
 '영화다': 13,
 '하지만': 14,
 '작품': 15,
 '별로': 16,
 '끝까지': 17,
 '영화의': 18,
 '아주': 19,
 '내내': 20,
 '같다': 21,
 '지금': 22,
 '최악의': 23,
 '좋다': 24,
 '시간': 25,
 '할': 26,
 '뭔가': 27,
 '다른': 28,
 '재미': 29,
 '있다': 30,
 '그래도': 31,
 '않고': 32,
 '재미있게': 33,
 '어떻게': 34,
 '많은': 35,
 '중': 36,
 '하나': 37,
 '연기가': 38,
 '별': 39,
 '감독': 40,
 '재밌다': 41,
 '굿': 42,
 '이걸': 43,
 '아니라': 44,
 '이야기': 45,
 '연기도': 46,
 '모두': 47,
 '봤다': 48,
 '아니다': 49,
 '근데': 50,
 '아름다운': 51,
 '때문에': 52,
 '스토리가': 53,
 '보다가': 54,
 '명작': 55,
 '나는': 56,
 '감동': 57,
 '계속': 58,
 '제대로': 59,
 '나오는': 60,
 '영화에': 61,
 '느낌': 62,
 '제일': 63,
 '내용도': 64,
 '모든': 65,
 '내용이': 66,
 '시간이': 67,
 '점도': 68,
 '않은': 69,
 '보다': 70,
 '차라리': 71,
 '아직도': 72,
 '액션': 73,
 '아니고': 74,
 '재미없다': 75,
 '아닌': 76,
 '감독이': 77,
 '년': 78,
 '감독의': 79,
 '스토리도': 80,
 '영화라고': 81,
 '않는': 82,
 '만드는': 83,
 '특히': 84,
 '여운이': 85,
 '한번': 86,
 '도대체': 87,
 '영화입니다':

In [55]:
vocab = FreqDist(np.hstack(my_words))    # 빈도수를 알아서 구해주는 함수와 클래스들
vocab_size = 500                          # top 500만 사용하기
vocab = vocab.most_common(vocab_size)     
print(f'단어 집합의 크기 : {len(vocab)}')
print('빈도수 상위 10선:',vocab[:10])

단어 집합의 크기 : 500
빈도수 상위 10선: [('최고의', 2231), ('드라마', 1861), ('완전', 1814), ('평점', 1805), ('좋은', 1745), ('평점이', 1614), ('연기', 1596), ('쓰레기', 1503), ('최고', 1451), ('스토리', 1412)]


In [56]:
vocab

[('최고의', 2231),
 ('드라마', 1861),
 ('완전', 1814),
 ('평점', 1805),
 ('좋은', 1745),
 ('평점이', 1614),
 ('연기', 1596),
 ('쓰레기', 1503),
 ('최고', 1451),
 ('스토리', 1412),
 ('아깝다', 1172),
 ('마지막', 1090),
 ('영화다', 1066),
 ('하지만', 970),
 ('작품', 948),
 ('별로', 946),
 ('끝까지', 946),
 ('영화의', 852),
 ('아주', 844),
 ('내내', 842),
 ('같다', 834),
 ('지금', 827),
 ('최악의', 817),
 ('좋다', 814),
 ('시간', 806),
 ('할', 805),
 ('뭔가', 805),
 ('다른', 797),
 ('재미', 785),
 ('있다', 775),
 ('그래도', 768),
 ('않고', 765),
 ('재미있게', 753),
 ('어떻게', 752),
 ('많은', 747),
 ('중', 746),
 ('하나', 745),
 ('연기가', 743),
 ('별', 735),
 ('감독', 713),
 ('재밌다', 712),
 ('굿', 711),
 ('이걸', 709),
 ('아니라', 698),
 ('이야기', 693),
 ('연기도', 692),
 ('모두', 689),
 ('봤다', 687),
 ('아니다', 686),
 ('근데', 683),
 ('아름다운', 678),
 ('때문에', 675),
 ('스토리가', 672),
 ('보다가', 672),
 ('명작', 666),
 ('나는', 663),
 ('감동', 660),
 ('계속', 657),
 ('제대로', 653),
 ('나오는', 650),
 ('영화에', 648),
 ('느낌', 647),
 ('제일', 644),
 ('내용도', 642),
 ('모든', 641),
 ('내용이', 636),
 ('시간이', 633),
 ('점도', 633),
 ('않은

In [57]:
result = [item[0] for item in vocab]

In [58]:
result

['최고의',
 '드라마',
 '완전',
 '평점',
 '좋은',
 '평점이',
 '연기',
 '쓰레기',
 '최고',
 '스토리',
 '아깝다',
 '마지막',
 '영화다',
 '하지만',
 '작품',
 '별로',
 '끝까지',
 '영화의',
 '아주',
 '내내',
 '같다',
 '지금',
 '최악의',
 '좋다',
 '시간',
 '할',
 '뭔가',
 '다른',
 '재미',
 '있다',
 '그래도',
 '않고',
 '재미있게',
 '어떻게',
 '많은',
 '중',
 '하나',
 '연기가',
 '별',
 '감독',
 '재밌다',
 '굿',
 '이걸',
 '아니라',
 '이야기',
 '연기도',
 '모두',
 '봤다',
 '아니다',
 '근데',
 '아름다운',
 '때문에',
 '스토리가',
 '보다가',
 '명작',
 '나는',
 '감동',
 '계속',
 '제대로',
 '나오는',
 '영화에',
 '느낌',
 '제일',
 '내용도',
 '모든',
 '내용이',
 '시간이',
 '점도',
 '않은',
 '보다',
 '차라리',
 '아직도',
 '액션',
 '아니고',
 '재미없다',
 '아닌',
 '감독이',
 '년',
 '감독의',
 '스토리도',
 '영화라고',
 '않는',
 '만드는',
 '특히',
 '여운이',
 '한번',
 '도대체',
 '영화입니다',
 '보면서',
 '오랜만에',
 '재미도',
 '딱',
 '싶다',
 '나도',
 '배우들',
 '마지막에',
 '연출',
 '아까운',
 '나름',
 '처음',
 '좋았다',
 '좋고',
 '지루하고',
 '개',
 '배우들의',
 '재밌어요',
 '봤습니다',
 '그런',
 '좋아하는',
 '진심',
 '짱',
 '주는',
 '이영화',
 '남는',
 '보기',
 '절대',
 '얼마나',
 '이제',
 '이해가',
 '요즘',
 '내용',
 '연기는',
 '없음',
 '보는내내',
 '뻔한',
 '영화로',
 '개인적으로',
 '조금',
 '영화도',
 '최고다',
 'ㅋㅋㅋㅋ',
 '년대',


In [59]:
# 문장 ==> 숫자로 변환
# sentences=['Today is a sunny day', 
        #    'Today is a rainy day',
        #    'Is it sunny today?']
rets=tokenizer.texts_to_sequences(result)
rets

[[1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25],
 [26],
 [27],
 [28],
 [29],
 [30],
 [31],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [44],
 [45],
 [46],
 [47],
 [48],
 [49],
 [50],
 [51],
 [52],
 [53],
 [54],
 [55],
 [56],
 [57],
 [58],
 [59],
 [60],
 [61],
 [62],
 [63],
 [64],
 [65],
 [66],
 [67],
 [68],
 [69],
 [70],
 [71],
 [72],
 [73],
 [74],
 [75],
 [76],
 [77],
 [78],
 [79],
 [80],
 [81],
 [82],
 [83],
 [84],
 [85],
 [86],
 [87],
 [88],
 [89],
 [90],
 [91],
 [92],
 [93],
 [94],
 [95],
 [96],
 [97],
 [98],
 [99],
 [100],
 [101],
 [102],
 [103],
 [104],
 [105],
 [106],
 [107],
 [108],
 [109],
 [110],
 [111],
 [112],
 [113],
 [114],
 [115],
 [116],
 [117],
 [118],
 [119],
 [120],
 [121],
 [122],
 [123],
 [124],
 [125],
 [126],
 [127],
 [128],
 [129],
 [130],
 [131],
 [132],
 [133],
 [134],
 [135],
 [136],
 [137],
 [138],
 [13

In [60]:
# 전체 문장의 길이 통일 ==> 패딩
lengths=[len(x) for x in rets]
TOKEN_LENTH=max(lengths)

In [61]:
# 전체문장의 길이 통일 ==> 패딩

rets2=pad_sequences(rets,maxlen=TOKEN_LENTH,padding='post',truncating='post')
rets2, rets2.shape, rets2.ndim

(array([[  1],
        [  2],
        [  3],
        [  4],
        [  5],
        [  6],
        [  7],
        [  8],
        [  9],
        [ 10],
        [ 11],
        [ 12],
        [ 13],
        [ 14],
        [ 15],
        [ 16],
        [ 17],
        [ 18],
        [ 19],
        [ 20],
        [ 21],
        [ 22],
        [ 23],
        [ 24],
        [ 25],
        [ 26],
        [ 27],
        [ 28],
        [ 29],
        [ 30],
        [ 31],
        [ 32],
        [ 33],
        [ 34],
        [ 35],
        [ 36],
        [ 37],
        [ 38],
        [ 39],
        [ 40],
        [ 41],
        [ 42],
        [ 43],
        [ 44],
        [ 45],
        [ 46],
        [ 47],
        [ 48],
        [ 49],
        [ 50],
        [ 51],
        [ 52],
        [ 53],
        [ 54],
        [ 55],
        [ 56],
        [ 57],
        [ 58],
        [ 59],
        [ 60],
        [ 61],
        [ 62],
        [ 63],
        [ 64],
        [ 65],
        [ 66],
        [ 

In [62]:
model = Sequential()

In [65]:
from tensorflow.keras.layers import SimpleRNN, RNN, LSTM, GRU
# 5개의 토큰이 하나의 문장=> 타입스텝프
# 단어 수만큼의 컬럼을 가짐 ==> 2개로 변환
# 토큰별 단어 수 만큼 행 1x9 ==> 1x2
#                (input_dim,   output_dim, input_length)
# model.add(Embedding(word_index+1,  2,   input_length=5))
# model.add(Input(shape=(3,5)))
# model.add(SimpleRNN(4))
# model.add(RNN())
# model.add(GRU())
model.add(LSTM(128))
model.add(Dense(32))

In [66]:
# 파라미터
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               68608     
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
Total params: 72,736
Trainable params: 72,736
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.compile(loss='mse')

In [68]:
out=model.predict(result)

RuntimeError: Data adapters should be mutually exclusive for handling inputs. Found multiple adapters [<class 'keras.engine.data_adapter.ListsOfScalarsDataAdapter'>, <class 'keras.engine.data_adapter.GenericArrayLikeDataAdapter'>] to handle input: (<class 'list'> containing values of types {"<class 'numpy.str_'>"}), <class 'NoneType'>